# Imports

In [5]:
!pip install qiskit

In [6]:
import numpy as np
from qiskit import QuantumCircuit

### Setup a quantum circuit for preparing the quantum state $|000> + i |111>$

In [7]:
qc_example = QuantumCircuit(3)
qc_example.h(0)  # generate superpostion
qc_example.p(np.pi / 2, 0)  # add quantum phase
qc_example.cx(0, 1)  # 0th-qubit-Controlled-NOT gate on 1st qubit
qc_example.cx(0, 2)  # 0th-qubit-Controlled-NOT gate on 2nd qubit

### Add the classical output in the form of measurement of all qubits

In [ ]:
qc_measured = qc_example.measure_all(inplace=False)

### Execute using the [Sampler](https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.Sampler) primitive

In [28]:
from qiskit.primitives.sampler import Sampler

sampler = Sampler()
job = sampler.run(qc_measured, shots=10**18)
result = job.result()
print(f" > Quasi probability distribution: {result.quasi_dists}")

### Define the observable to be measured 

In [8]:
from qiskit.quantum_info import SparsePauliOp

operator = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])

### Execute using the Estimator primitive

In [15]:
from qiskit.primitives import Estimator

estimator = Estimator()
job = estimator.run(qc_example, operator, shots=100)
result = job.result()
print(f" > Expectation values: {result.values}")

In [20]:
def t(shots):
    estimator = Estimator()
    job = estimator.run(qc_example, operator, shots=shots)
    return job.result()

In [24]:
# Generate measurement examples

In [41]:
import pandas as pd

pd.set_option("display.max_columns", 500)
n = 1000
data = pd.DataFrame(
    {"measurements": [t(shots=i).values[0] for i in range(1, n)], "shots": range(1, n)}
)

# Plot data distribution

In [45]:
import seaborn as sns

sns.set_theme(style="white", color_codes=True)
mpg = sns.load_dataset("mpg")

# Use JointGrid directly to draw a custom plot
g = sns.JointGrid(data=data, x="shots", y="measurements", space=0, ratio=20)
g.plot_joint(sns.scatterplot, color="g", alpha=0.6, legend=False)
g.plot_marginals(sns.rugplot, height=1, color="g", alpha=0.6)

In [43]:
data.T

In [46]:
data["measurements"][0]